# Assignment 7

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [2]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [3]:
def expected_return(P, rf, mkt_rp):
    total_investment = P['Amount Invested'].sum()
    P['Weight'] = P['Amount Invested'] / total_investment
    P['Return'] = (rf + P['Beta'] * mkt_rp) * P['Weight']
    return P['Return'].sum()

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [4]:
def volatility(P, cor):
    total_investment = P['Amount Invested'].sum()
    P['Weight'] = P['Amount Invested'] / total_investment
    n = len(P)
    var = 0
    for i in range(n) :         # uses "variance = sum of covariances" formula
        for j in range(n) : 
            if (i == j) :
                var += P.iloc[i, 3] ** 2 * P.iloc[i, 1] ** 2
            else :
                var += P.iloc[i, 3] * P.iloc[j, 3] * cor * P.iloc[i, 1] * P.iloc[j, 1]

    return np.sqrt(var) 

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [5]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    return vol / mkt_vol * mkt_rp + rf

### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [80]:
def CAPM_beta(return_path, factors_path):
    df1 = pd.read_csv(return_path) 
    df2 = pd.read_csv(factors_path)

    df1['date'] = df1['year'] * 100 + df1['month']
    df1 = df1.drop('year', axis=1)
    df1 = df1.drop('month', axis=1)
    df2['Mkt-RF'] /= 100
    df2['RF'] /= 100
    data = pd.merge(df1, df2, on='date', how='left')
    data["R_i-Rf"] = data['RET'] - data['RF']

    groups = data.groupby(["CUSIP"], group_keys = False)
    exp_ret = groups['RET'].mean().reset_index()
    cov = groups.apply(lambda group: np.cov(group['R_i-Rf'], group['Mkt-RF'])[0, 1])
    var = groups["Mkt-RF"].var()
    beta = cov / var
    beta.name = 'BETA'

    cusip = list(data['CUSIP'].unique())
    output = pd.DataFrame(data = {
        "CUSIP" : cusip,
        # "EXP_RETURN" : [],
        # "BETA" : []
    })

    output = output.merge(exp_ret, on='CUSIP')
    output = output.merge(beta, on='CUSIP')
    output.rename(columns={'RET': 'EXP_RETURN'}, inplace=True)

    return output

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [86]:
def slope(data):
    # This should return 0.007981588120535642... with the provided data.
    coefficients = np.polyfit(data['BETA'], data['EXP_RETURN'], 1)
    return coefficients[1]